## **Optimization**
---

>### Gradient Descent
- **Goal**: Optimize the Rosenbrock function (classic test function used in optimization problems)  
$$(1 - x)^2 + 100 * (y - x^2)^2$$
- **Key Takeaways**
    - create model: `Model(Ipopt.Optimizer) => register(model, :R, 2, R; autodiff = true)`
    - define variables: `@variable(model, -20 <= x <= 20, start = -20)`
    - define objective function: `@NLobjective(model, Min, R(x,y))`
    - (solver options: `set_optimizer_attribute(...)`)
    - solve optimization: `optimize!(model)`


Packages used:
<span style="color:yellow; background-color:green">JuMP, Ipopt</span>

In [ ]:
using JuMP, Ipopt

In [13]:
R(x,y) = (1-x)^2 + 100 * (y-x^2)^2

# Create a new model with Ipopt as the solver
model = Model(Ipopt.Optimizer)

# Manually register the function with the model to use automatic differentiation
register(model, :R, 2, R; autodiff = true)

# Define variables & initial guess
@variable(model, -20 <= x <= 20, start = -20)
@variable(model, -20 <= y <= 20, start = 20)

# No initial guess
# @variable(model, -20 <= x <= 20)
# @variable(model, -20 <= y <= 20)

# Find the wrong minimum
# @variable(model, -20 <= x <= 20, start = -1.2)
# @variable(model, -20 <= y <= 20, start = 1)


# Define objective function: Rosenbrock function
@NLobjective(model, Min, R(x,y))

# Solver options
set_optimizer_attribute(model, "max_iter", 1000)    # Increase max iterations
set_optimizer_attribute(model, "tol", 1e-9)         # tighter convergence tolerance
set_optimizer_attribute(model, "print_level", 0)    # suppress all of the output => set to 0 (4 is compromised)

# Solve the optimization problem
optimize!(model)

# Display results
println("Optimal parameters (x, y): (", value(x), ", ", value(y), ")")
println("Minimum objective value: ", objective_value(model))

Optimal parameters (x, y): (0.9999999837814544, 0.9999999677255611)
Minimum objective value: 2.656867929696428e-16


>### Gradient Descent - Multivariable example
- **Goal**: Optimize Himmelblau's function (classic test function used in optimization problems)  
$$f(x, y) = (x^2 + y - 11)^2 + (x + y^2 - 7)^2$$
- **Key Takeaways**
    - solve optimization using previous method
    - compute hessian: `FiniteDiff.finite_difference_hessian(himmelblau_wrapper, [xStar, yStar])`
    - compute eigenvalues: `eigvals(hessian)`
    - Determine critical point's nature

Packages used:
<span style="color:yellow; background-color:green">JuMP, Ipopt, FiniteDiff, LinearAlgebra</span>

In [15]:
using JuMP, Ipopt, FiniteDiff, LinearAlgebra

In [18]:
# Himmelblau function definition remains the same
function himmelblau(x, y)
    return (x^2 + y - 11)^2 + (x + y^2 - 7)^2
end

# Extended function to also compute Hessian and its eigenvalues
function optimize_and_analyze_himmelblau(x_init, y_init)
    model = Model(Ipopt.Optimizer)
    
    @variable(model, x, start = x_init)
    @variable(model, y, start = y_init)
    
    @NLobjective(model, Min, (x^2 + y - 11)^2 + (x + y^2 - 7)^2)
    
    set_optimizer_attribute(model, "max_iter", 1000)
    set_optimizer_attribute(model, "tol", 1e-9)
    set_optimizer_attribute(model, "print_level", 0)
    
    optimize!(model)
    
    xStar, yStar = value(x), value(y)
    println("======================================")
    println("Optimal parameters (x, y): (", xStar, ", ", yStar, ")")
    println("Minimum objective value: ", objective_value(model))
    
    # Define a wrapper function for FiniteDiff
    function himmelblau_wrapper(v)
        return himmelblau(v[1], v[2])
    end

    # Compute hessian and eigenvalues
    hessian = FiniteDiff.finite_difference_hessian(himmelblau_wrapper, [xStar, yStar])
    eigenvalues = eigvals(hessian)

    println("Hessian at the optimal point: \n", hessian)
    println("Eigenvalues of the Hessian: ", eigenvalues)
    
    # Determine the nature of the critical point based on the eigenvalues
    if all(eig -> eig > 0, eigenvalues)
        println("The point is a local minimum.")
    elseif all(eig -> eig < 0, eigenvalues)
        println("The point is a local maximum.")
    else
        println("The point is a saddle point or the test is inconclusive.")
    end
    println("======================================")
end

optimize_and_analyze_himmelblau (generic function with 1 method)

In [20]:
# Starting points near the four local minima
starting_points = [(3, 2), (-2.805118, 3.283186), (-3.779310, -3.283186), (3.584428, -1.848126)]

# Optimize Himmelblau's function and analyze each result from the starting points
for (x_init, y_init) in starting_points
    println("Starting from point: (", x_init, ", ", y_init, ")")
    optimize_and_analyze_himmelblau(x_init, y_init)
end

Starting from point: (3, 2)
Optimal parameters (x, y): (3.0, 2.0)
Minimum objective value: 0.0
Hessian at the optimal point: 
[74.0000002682209 20.0; 20.0 34.00000011920929]
Eigenvalues of the Hessian: [25.715728893569633, 82.28427149386056]
The point is a local minimum.
Starting from point: (-2.805118, 3.283186)
Optimal parameters (x, y): (-2.805118086952745, 3.131312518250573)
Minimum objective value: 7.888609052210118e-31
Hessian at the optimal point: 
[64.949500088456 1.3047777251870691; 1.3047777251870691 80.44094498775873]
Eigenvalues of the Hessian: [64.84037300554613, 80.5500720706686]
The point is a local minimum.
Starting from point: (-3.77931, -3.283186)
Optimal parameters (x, y): (-3.779310253377774, -3.2831859912861767)
Minimum objective value: 3.996369345849646e-26
Hessian at the optimal point: 
[116.26548835580718 -28.24998497866954; -28.24998497866954 88.23448234835634]
Eigenvalues of the Hessian: [70.71435509450659, 133.78561560965693]
The point is a local minimum.
Sta

>### Constrained Optimization - Lagrange's Method
- **Goal**: Optimize the cost function under two constraint equations
$$
Cost(x,y,z) = -(x+y+z) \\
g_1 = x^2 + y^2 -1 \\
g_2 = x+z-2
$$   

- **Key Takeaways**
    - symbolic systems (defining)
    - Lagrangian function: $L(x,y,z) = Cost(x,y,z) + \lambda_1 * g_1(x,y,z) + \lambda_2 * g_2(x,y,z)$
    - compute gradient: `Symbolics.gradient(L, [var;lam])`
    - solve for gradL = 0: `nlsolve(gradL_num, initial_guess)`
    - process the solutions to find minimum
- **JuMP & Ipopt application**

Packages used:
<span style="color:yellow; background-color:green">Symbolics, NLsolve, Random</span>

In [21]:
using Symbolics, NLsolve, Random

In [22]:
#=====================SYMBOLIC SOLUTION APPROACH=======================#

# Define symbolic variables
@Symbolics.variables var[1:3]
@Symbolics.variables lam[1:2]

# Define cost function
Cost = -(var[1] + var[2] + var[3])

# Define constraint equations
g1 = var[1]^2 + var[2]^2 - 1
g2 = var[1] + var[3] - 2

# Formulate the Lagrangian: objective function + lagrange multipliers for constraints
L = Cost + lam[1]*g1 + lam[2]*g2

# Compute the gradient
gradL = Symbolics.gradient(L, [var;lam])

display(gradL)

5-element Vector{Num}:
 -1 + lam[2] + 2lam[1]*var[1]
          -1 + 2lam[1]*var[2]
                  -1 + lam[2]
     -1 + var[1]^2 + var[2]^2
         -2 + var[1] + var[3]

In [23]:
#=====================NUMERICAL SOLUTION APPROACH=======================#

# Convert symbolic expression to numerical function
gradL_num = build_function(gradL, [var; lam])
gradL_num = eval(gradL_num[1])

# Initialize an array to store solutions
cost_solutions = Float64[]

# Solve with multiple initial guesses
for k = 1:100
    initial_guess = randn(5)
    result = nlsolve(gradL_num, initial_guess, xtol=1e-6, ftol=1e-7)

    # Process converged solutions
    if converged(result)
        var_val = result.zero[1:3]
        lam_val = result.zero[4:5]
        cost_extremePoint = -sum(var_val)

        # Add the found solution if sufficiently diff from previous found solution
        if all(abs(cost_extremePoint-sol) > 1e-2 for sol in cost_solutions)
            push!(cost_solutions, cost_extremePoint)
            @show [var_val; lam_val]
        end
    end
end

sorted_solutions = sort(cost_solutions)

[var_val; lam_val] = [1.8163521787042063e-11, -1.0000000000098037, 1.9999999999818365, -0.49999999998886063, 1.0]
[var_val; lam_val] = [1.5028135762532895e-14, 1.0000000000000162, 1.999999999999985, 0.4999999999999901, 1.0]


2-element Vector{Float64}:
 -3.000000000000016
 -0.9999999999901963

In [30]:
#=====================SOLVE USING JuMP=======================#
model = Model(Ipopt.Optimizer)
@variable(model, var[1:3])
@NLobjective(model, Min, -(var[1]+var[2]+var[3]))

# Add constraint equations
@NLconstraint(model, var[1]^2 + var[2]^2 == 1)
@NLconstraint(model, var[1] + var[3] == 2)

set_optimizer_attribute(model, "max_iter", 1000)
set_optimizer_attribute(model, "tol", 1e-9)
set_optimizer_attribute(model, "print_level", 0)
    
optimize!(model)

# Display the results
println("Objective value: ", objective_value(model))
println("Solution: ")
println("var[1] = ", value(var[1]))
println("var[2] = ", value(var[2]))
println("var[3] = ", value(var[3]))

Objective value: -3.0
Solution: 
var[1] = 4.232217986118638e-17
var[2] = 1.0
var[3] = 2.0


>### Constrained gradient descent
- **Goal**: Optimize the cost function under equality constraints
$$
Cost(x_1,x_2,x_3,x_4) = x_1^2 + x_2^2 + x_3^2 + x_4^2\\
G_1(x_1,x_2,x_3,x_4) = x_1 + x_2 - 3\\ 
G_2(x_1,x_2,x_3,x_4) = sin(\pi x_1) - x_3^2 - 1 + x_4^3
$$   

- **Requirements**
    - Decreasing direction: $\nabla f(x_k) * \Delta x < 0$
    - Following constraints: $\nabla g_i(x_k) * \Delta x = 0$
    - Formulas used (remove components of $\nabla g_i(x_k)$ from $\nabla f(x_k)$):
$$
\mathrm{proj}_{\mathbf{b}} \mathbf{a} = \left( \frac{\mathbf{a} \cdot \mathbf{b}}{\mathbf{b} \cdot \mathbf{b}} \right) \mathbf{b}\\
\Delta x = - \nabla f(x_k) + \sum_{i=1}^{m} a_i\nabla g_i(x_k)\
$$
- **Key Takeaways**
    - symbolic systems (defining)
    - Lagrangian function: $L(x,y,z) = Cost(x,y,z) + \lambda_1 * g_1(x,y,z) + \lambda_2 * g_2(x,y,z)$
    - compute gradient: `Symbolics.gradient(L, [var;lam])`
    - solve for gradL = 0: `nlsolve(gradL_num, initial_guess)`
    - process the solutions to find minimum
- **JuMP & Ipopt application**

Packages used:
<span style="color:yellow; background-color:green">ForwardDiff, LinearAlgebra, NLsolve</span>

In [44]:
using ForwardDiff, LinearAlgebra, NLsolve

In [45]:
# Define the cost function f(x) and the equality constraints G(x)
function cost(x)
    return x[1]^2 + x[2]^2 + x[3]^2 + x[4]^2
end
 
function G(x)
    return [x[1] + x[2] - 3, sin(pi*x[1]) - x[3]^2 - 1 + x[4]^3]
end
 
# Compute the gradient of f and the Jacobian of G using ForwardDiff
grad_cost(x) = ForwardDiff.gradient(cost, x)
JacG(x) = ForwardDiff.jacobian(G, x)

JacG (generic function with 1 method)

In [46]:
#=================================== FIND THE ORTHONORMAL BASIS OF JACOBIAN_G =====================================#
function gram_schmidt(jacobian_G)
    n, m = size(jacobian_G)
    B = zeros(n,m)  # Basis orthonormal Vectors to be returned

    for j=1:m
        # start with original vectors
        b = jacobian_G[:, j]    
        
        for i = 1:j-1
            b -= dot(B[:,i], jacobian_G[:, j]) * B[:,i]
        end
        B[:,j] = b / norm(b)
    end
    
    return B
end

gram_schmidt (generic function with 1 method)

In [52]:
#======================== PROJECT THE GRADIENT ONTO THE FEASIBLE DIRECTIONS =================================#
function project_grad_cost(grad_cost, B)
    # Iterate over columns of B, subtracting their projection from grad_f
    projected_grad_cost = grad_cost
    for j = 1:size(B, 2)
        projected_grad_cost = projected_grad_cost - dot(grad_cost, B[:,j]) * B[:,j]     # vectors in B are already orthonormal, so no need to divide by norm(B[:,j])
    end
    return projected_grad_cost
end

project_grad_cost (generic function with 1 method)

In [48]:
# Find feasible starting point using NLsolve
function find_feasible_start(G, x_guess)
    function constraint!(F, x)
        F[1:2] = G(x)
    end
    xFeasible = nlsolve(constraint!, x_guess)
    return xFeasible.zero
end

find_feasible_start (generic function with 1 method)

In [53]:
# Actual algorithm
x_guess = [0.0, 0.0, 0.0, 0.0] # Not feasible
x0 = find_feasible_start(G, x_guess) # Make feasible
  
# set step size, maxIter, tolerances
s=0.1
maxIter=10000
gradTol = 1e-6
constraintTol = 1e-4
 
xk = x0

for k=1:maxIter
    jacobian_G_transposed = transpose(JacG(xk))
    B = gram_schmidt(jacobian_G_transposed)
    projected_grad_cost = project_grad_cost(grad_cost(xk), B)
    if norm(projected_grad_cost) < gradTol
        println("---- Iteration $k ----")
        println("Met tolerances.")
        println("JacCost applied to null(JacG) (should be near 0): ", 
           round.((grad_cost(xk)')*nullspace(JacG(xk)), digits=5))
        println("Constraints (should be near 0): ", round.(G(xk), digits=5))
        println("----------------- ")
        break
    else
        # Update
        xk = xk - s*projected_grad_cost
    end
    if norm(G(xk)) > constraintTol
        xk = find_feasible_start(G, xk) # Again Feasible
    end
end
xStar = xk
costStar = cost(xStar)

println("\nFinal Results:")
println("-----------------")
println("Optimal x: ", round.(xStar, digits=5))

println("Optimal cost(xStar): ", round(costStar, digits=5))
 

---- Iteration 184 ----
Met tolerances.
JacCost applied to null(JacG) (should be near 0): [0.0 -0.0]
Constraints (should be near 0): [-0.0, 2.0e-5]
----------------- 

Final Results:
-----------------
Optimal x: [0.95451, 2.04549, 0.0, 0.95008]
Optimal cost(xStar): 5.99778
